In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv
/kaggle/input/alx-movie-recommendation-project-2024/movies.csv
/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv
/kaggle/input/alx-movie-recommendation-project-2024/train.csv
/kaggle/input/alx-movie-recommendation-project-2024/test.csv
/kaggle/input/alx-movie-recommendation-project-2024/tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/links.csv


**Data Exploration and Understanding**

In [2]:
import numpy as np
import pandas as pd
import os

# List all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load the datasets
train_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
test_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')
movies_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/movies.csv')
tags_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/tags.csv')
genome_scores_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv')
genome_tags_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv')
imdb_data_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv')
links_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/links.csv')

# Inspect the data
print(train_df.head())
print(test_df.head())
print(movies_df.head())
print(tags_df.head())
print(genome_scores_df.head())
print(genome_tags_df.head())
print(imdb_data_df.head())
print(links_df.head())


/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv
/kaggle/input/alx-movie-recommendation-project-2024/movies.csv
/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv
/kaggle/input/alx-movie-recommendation-project-2024/train.csv
/kaggle/input/alx-movie-recommendation-project-2024/test.csv
/kaggle/input/alx-movie-recommendation-project-2024/tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/links.csv
   userId  movieId  rating   timestamp
0    5163    57669     4.0  1518349992
1  106343        5     4.5  1206238739
2  146790     5459     5.0  1076215539
3  106362    32296     2.0  1423042565
4    9041      366     3.0   833375837
   userId  movieId
0       1     2011
1       1     4144
2       1     5767
3       1     6711
4       1     7318
   movieId                               title  \
0        1     

**Data Preprocessing**

In [3]:


# Handle missing values in train data
train_df.fillna(train_df.mean(), inplace=True)
print("\nTrain DataFrame after filling missing values:")
print(train_df.head())

# Merge train data with movies data
train_merged = pd.merge(train_df, movies_df, on='movieId', how='left')
print("\nMerged Train DataFrame Head:")
print(train_merged.head())



Train DataFrame after filling missing values:
   userId  movieId  rating   timestamp
0    5163    57669     4.0  1518349992
1  106343        5     4.5  1206238739
2  146790     5459     5.0  1076215539
3  106362    32296     2.0  1423042565
4    9041      366     3.0   833375837

Merged Train DataFrame Head:
   userId  movieId  rating   timestamp  \
0    5163    57669     4.0  1518349992   
1  106343        5     4.5  1206238739   
2  146790     5459     5.0  1076215539   
3  106362    32296     2.0  1423042565   
4    9041      366     3.0   833375837   

                                               title  \
0                                   In Bruges (2008)   
1                 Father of the Bride Part II (1995)   
2  Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...   
3     Miss Congeniality 2: Armed and Fabulous (2005)   
4  Wes Craven's New Nightmare (Nightmare on Elm S...   

                          genres  
0    Comedy|Crime|Drama|Thriller  
1                         Com

**Feature Engineering**

In [4]:
# Calculate user and movie average ratings
user_avg_rating = train_df.groupby('userId')['rating'].mean()
movie_avg_rating = train_df.groupby('movieId')['rating'].mean()

print("\nUser Average Ratings:")
print(user_avg_rating.head())
print("\nMovie Average Ratings:")
print(movie_avg_rating.head())

# Join user and movie average ratings with train_merged
train_merged = train_merged.join(user_avg_rating, on='userId', rsuffix='_user')
train_merged = train_merged.join(movie_avg_rating, on='movieId', rsuffix='_movie')

print("\nTrain Merged DataFrame with User and Movie Average Ratings:")
print(train_merged.head())


User Average Ratings:
userId
1    3.910714
2    3.416667
3    3.691235
4    3.308989
5    3.885714
Name: rating, dtype: float64

Movie Average Ratings:
movieId
1    3.889971
2    3.263414
3    3.132325
4    2.878099
5    3.059165
Name: rating, dtype: float64

Train Merged DataFrame with User and Movie Average Ratings:
   userId  movieId  rating   timestamp  \
0    5163    57669     4.0  1518349992   
1  106343        5     4.5  1206238739   
2  146790     5459     5.0  1076215539   
3  106362    32296     2.0  1423042565   
4    9041      366     3.0   833375837   

                                               title  \
0                                   In Bruges (2008)   
1                 Father of the Bride Part II (1995)   
2  Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...   
3     Miss Congeniality 2: Armed and Fabulous (2005)   
4  Wes Craven's New Nightmare (Nightmare on Elm S...   

                          genres  rating_user  rating_movie  
0    Comedy|Crime|Drama|Thr

**Building the Model**

In [5]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load data into Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)

# Model training
model = SVD()
model.fit(trainset)

# Predictions
predictions = model.test(testset)

# Evaluation
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

RMSE: 0.8333
RMSE: 0.8333175842306644


**Model Training and Prediction**

In [6]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import GridSearchCV, train_test_split
import pandas as pd

# Create a sample DataFrame
data = {
    'userId': [1, 2, 3, 4, 5],
    'movieId': [10, 20, 30, 40, 50],
    'rating': [4.0, 3.5, 2.0, 5.0, 4.5]
}
train_df = pd.DataFrame(data)

# Load data into the Surprise Dataset
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], Reader(rating_scale=(0.5, 5.0)))

# Parameter tuning grid
param_grid = {
    'n_factors': [20, 50],
    'n_epochs': [20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

# Perform grid search
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=1)
gs.fit(data)

# Best parameters
best_params = gs.best_params['rmse']
print(f'Best parameters: {best_params}')

# Train with best parameters on full dataset
best_model = SVD(**best_params)
trainset = data.build_full_trainset()
best_model.fit(trainset)

# Create a sample test DataFrame
test_data = {
    'userId': [1, 2, 3, 4, 5],
    'movieId': [10, 20, 30, 40, 50],
    'rating': [4.0, 3.5, 2.0, 5.0, 4.5]
}
test_df = pd.DataFrame(test_data)

# Load test data into the Surprise Dataset
testset = Dataset.load_from_df(test_df[['userId', 'movieId', 'rating']], Reader(rating_scale=(0.5, 5.0))).build_full_trainset().build_testset()

# Make predictions
predictions = best_model.test(testset)

# Calculate and print RMSE
rmse = accuracy.rmse(predictions)

# Adding the predicted ratings to the test DataFrame
test_df['predicted_rating'] = [pred.est for pred in predictions]

# Display the first few rows with predicted ratings
print(test_df.head())


Best parameters: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.02}
RMSE: 0.9428
   userId  movieId  rating  predicted_rating
0       1       10     4.0          3.793649
1       2       20     3.5          3.790023
2       3       30     2.0          3.647934
3       4       40     5.0          3.893881
4       5       50     4.5          3.884423


**Submission**

In [7]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import GridSearchCV

# Load train and test data
train_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
test_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')

# Load data into Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Parameter tuning
param_grid = {
    'n_factors': [20, 50],
    'n_epochs': [20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=1)
gs.fit(data)

best_params = gs.best_params['rmse']
print(f'Best parameters: {best_params}')

# Train with best parameters
best_model = SVD(**best_params)
best_model.fit(data.build_full_trainset())

# Efficient prediction
user_movie_pairs = list(zip(test_df['userId'], test_df['movieId']))
predictions = [best_model.predict(uid, mid).est for (uid, mid) in user_movie_pairs]
test_df['rating'] = predictions

# Format predictions
test_df['Id'] = test_df['userId'].astype(str) + '_' + test_df['movieId'].astype(str)
submission_df = test_df[['Id', 'rating']]

# Save to CSV
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print(submission_df.head())


Best parameters: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
       Id    rating
0  1_2011  3.037095
1  1_4144  4.231037
2  1_5767  3.570826
3  1_6711  4.014753
4  1_7318  3.148033
